In [3]:
import csv
import requests

In [4]:
# your AgroDataCube access token
token = "<your token here>"

In [5]:
# token to use for AgroDataCube requests
header_token = {"token":token}

# process all points in the file
with open('points.txt', newline='') as csvfile:
    pointsreader = csv.reader(csvfile, delimiter=';', quotechar='"')
    next(pointsreader) # skip header
    for row in pointsreader:
        id = row[1]
        x = row[2]
        y = row[3]
        
        # read all available fields for the point, for all available years
        fields_url = "http://agrodatacube.wur.nl/api/v1/rest/fields?epsg=28992&geometry=POINT({0} {1})&page_size=1000&page_offset=0".format(x, y)

        fields_data = requests.get(fields_url, headers=header_token)
        fields = fields_data.json()

        for field in fields["features"]:
            # basic field data
            field_id = field["properties"]["fieldid"]
            field_year = field["properties"]["year"]
            field_crop = field["properties"]["crop_name"]
            
            # average height for the field
            ahn_url = "http://agrodatacube.wur.nl/api/v1/rest/fields/{0}/ahn".format(field_id)
            ahn_data = requests.get(ahn_url, headers=header_token)
            ahn = ahn_data.json()
            field_ahn_cell_count = ahn["features"][0]["properties"]["count"]
            field_mean_height = 'NA'
            if field_ahn_cell_count != "0":
                field_mean_height = ahn["features"][0]["properties"]["mean"]
            
            # soiltype
            soiltypes_url = "http://agrodatacube.wur.nl/api/v1/rest/fields/{0}/soiltypes".format(field_id)
            soiltypes_data = requests.get(soiltypes_url, headers=header_token)
            soiltypes = soiltypes_data.json()
            soilcodes = [f["properties"]["soilcode"] for f in soiltypes["features"]]

            # ndvi
            ndvi_url = "http://agrodatacube.wur.nl/api/v1/rest/fields/{0}/ndvi?fromdate={1}&todate={2}".format(field_id, field_year, field_year)
            ndvi_data = requests.get(ndvi_url, headers=header_token)
            ndvi_timeseries = ndvi_data.json()
            ndvi = [f["properties"]["ndvi_value"] for f in ndvi_timeseries["features"]]
    
            # list results, probably want to create an output file from it ...
            print(id, ";", x, ";", y, ";", field_year, ";", field_id, ";", field_mean_height, ";", field_crop, ";", soilcodes, ";", ndvi)


1 ; 220390.66130215 ; 451315.159267097 ; 2016 ; 576448 ; 1365.391304347826 ; Mais, corncob mix ; ['Hn21'] ; [0.281347, 0.308512, 0.319994, 0.323977, 0.324712]
1 ; 220390.66130215 ; 451315.159267097 ; 2014 ; 1100526 ; 1365.391304347826 ; Gerst, zomer- ; ['Hn21'] ; [0.404959, 0.406094, 0.462494, 0.519188, 0.559018]
1 ; 220390.66130215 ; 451315.159267097 ; 2013 ; 1951155 ; 1365.391304347826 ; Mais, corncob mix ; ['Hn21'] ; [0.201676, 0.266229, 0.227812, 0.251935, 0.237418]
1 ; 220390.66130215 ; 451315.159267097 ; 2012 ; 2503421 ; 1365.391304347826 ; Aardappelen, consumptie- op zand-/veengrond ; ['Hn21'] ; []
1 ; 220390.66130215 ; 451315.159267097 ; 2015 ; 3532555 ; 1365.391304347826 ; Bieten, suiker- ; ['Hn21'] ; [0.175382, 0.274912, 0.200812, 0.200441, 0.303859]
1 ; 220390.66130215 ; 451315.159267097 ; 2017 ; 4407912 ; 1365.391304347826 ; Mais, snij- ; ['Hn21'] ; [0.291, 0.309, 0.177, 0.192, 0.156]
2 ; 220208.399914748 ; 450719.078264784 ; 2016 ; 778011 ; 1412.0851063829787 ; Grasland, b

11 ; 220277.795577844 ; 451619.537763465 ; 2015 ; 3998591 ; 1404.6923076923076 ; Mais, snij- ; ['Hn21'] ; [0.240133, 0.353993, 0.27638, 0.293787, 0.391933]
11 ; 220277.795577844 ; 451619.537763465 ; 2017 ; 4762564 ; 1404.6923076923076 ; Mais, snij- ; ['Hn21'] ; [0.252, 0.27, 0.225, 0.228, 0.237]
12 ; 222563.866419577 ; 452404.575958331 ; 2016 ; 180373 ; 1649.4594594594594 ; Mais, snij- ; ['cHn21', 'zEZ21'] ; [0.314575, 0.329075, 0.347754, 0.334687, 0.35735]
12 ; 222563.866419577 ; 452404.575958331 ; 2014 ; 882649 ; 1647.5263157894738 ; Mais, snij- ; ['cHn21', 'zEZ21'] ; [0.339713, 0.290404, 0.327617, 0.364138, 0.386654]
12 ; 222563.866419577 ; 452404.575958331 ; 2013 ; 2338301 ; 1647.5263157894738 ; Mais, snij- ; ['cHn21', 'zEZ21'] ; [0.24775, 0.314046, 0.276467, 0.283633, 0.285763]
12 ; 222563.866419577 ; 452404.575958331 ; 2012 ; 3104982 ; 1647.5263157894738 ; Mais, snij- ; ['cHn21', 'zEZ21'] ; []
12 ; 222563.866419577 ; 452404.575958331 ; 2015 ; 3994434 ; 1647.5263157894738 ; Mais, 

21 ; 220630.692490738 ; 451563.248895321 ; 2016 ; 180174 ; 1366.4285714285713 ; Grasland, blijvend ; ['Hn21', 'pZg23'] ; [0.52895, 0.5572, 0.57875, 0.60325, 0.5035]
21 ; 220630.692490738 ; 451563.248895321 ; 2015 ; 3593617 ; 1366.4285714285713 ; Grasland, blijvend ; ['Hn21', 'pZg23'] ; [0.45535, 0.4922, 0.40285, 0.47895, 0.5701]
21 ; 220630.692490738 ; 451563.248895321 ; 2017 ; 4098132 ; 1366.4285714285713 ; Grasland, blijvend ; ['Hn21', 'pZg23'] ; [0.597, 0.69, 0.693, 0.468, 0.423]
22 ; 221513.415883522 ; 452126.069558517 ; 2016 ; 231025 ; 1451.5652173913043 ; Mais, snij- ; ['cHn21', 'Hn21'] ; [0.42324, 0.45942, 0.421613, 0.46152, 0.48874]
22 ; 221513.415883522 ; 452126.069558517 ; 2014 ; 1526599 ; 1451.5652173913043 ; Mais, snij- ; ['cHn21', 'Hn21'] ; [0.376317, 0.368733, 0.37904, 0.413387, 0.465353]
22 ; 221513.415883522 ; 452126.069558517 ; 2013 ; 2338812 ; 1451.5652173913043 ; Mais, snij- ; ['cHn21', 'Hn21'] ; [0.31694, 0.39404, 0.26092, 0.3176, 0.29716]
22 ; 221513.415883522 ; 45

31 ; 221961.864129678 ; 451649.941337348 ; 2013 ; 1713868 ; 1508.7962962962963 ; Grasland, blijvend ; ['cHn21', 'Hn21', 'pZg23'] ; [0.45558, 0.61856, 0.552928, 0.56023, 0.570947]
31 ; 221961.864129678 ; 451649.941337348 ; 2012 ; 3129249 ; 1508.7962962962963 ; Grasland, blijvend ; ['cHn21', 'Hn21', 'pZg23'] ; []
31 ; 221961.864129678 ; 451649.941337348 ; 2015 ; 3365919 ; 1508.7962962962963 ; Grasland, blijvend ; ['cHn21', 'Hn21', 'pZg23'] ; [0.487202, 0.561167, 0.50862, 0.43645, 0.681555]
31 ; 221961.864129678 ; 451649.941337348 ; 2017 ; 4111088 ; 1508.7962962962963 ; Grasland, tijdelijk ; ['cHn21', 'Hn21', 'pZg23'] ; [0.654, 0.705, 0.69, 0.657, 0.666]
32 ; 222889.809670378 ; 452017.548525638 ; 2016 ; 490032 ; 1680.95 ; Erwten, groene/gele, groen te oogsten ; ['zEZ21'] ; [0.378991, 0.405213, 0.407174, 0.419287, 0.418135]
32 ; 222889.809670378 ; 452017.548525638 ; 2014 ; 1306764 ; 1688.2978723404256 ; Groenten open grond (inclusief groentezaden) ; ['cHn21', 'zEZ21'] ; [0.671798, 0.655196

40 ; 222197.713361785 ; 452211.810609801 ; 2015 ; 3424104 ; 1555.6933333333334 ; Grasland, blijvend ; ['Hn21', 'zEZ21'] ; [0.511783, 0.548867, 0.503123, 0.539035, 0.673771]
40 ; 222197.713361785 ; 452211.810609801 ; 2017 ; 4559217 ; 1555.6933333333334 ; Grasland, blijvend ; ['Hn21', 'zEZ21'] ; [0.57, 0.666, 0.648, 0.678, 0.681]
41 ; 220753.560858631 ; 450774.82325961 ; 2016 ; 184943 ; 1489.5238095238096 ; Mais, snij- ; ['Hn21'] ; [0.37578, 0.34929, 0.39317, 0.42361, 0.39424]
41 ; 220753.560858631 ; 450774.82325961 ; 2014 ; 1577910 ; 1489.5238095238096 ; Mais, snij- ; ['Hn21'] ; [0.40791, 0.3449, 0.40356, 0.43803, 0.47202]
41 ; 220753.560858631 ; 450774.82325961 ; 2013 ; 2373100 ; 1479.3846153846155 ; Mais, snij- ; ['Hn21'] ; [0.354683, 0.404433, 0.358017, 0.367667, 0.395417]
41 ; 220753.560858631 ; 450774.82325961 ; 2012 ; 3053682 ; 1478.6666666666667 ; Mais, snij- ; ['Hn21'] ; []
41 ; 220753.560858631 ; 450774.82325961 ; 2015 ; 3918091 ; 1489.5238095238096 ; Mais, snij- ; ['Hn21'] ; [

...

200 ; 221430.888886427 ; 452154.592091888 ; 2016 ; 103550 ; 1446.8235294117646 ; Grasland, blijvend ; ['Hn21'] ; [0.55975, 0.627006, 0.625013, 0.662656, 0.582019]
200 ; 221430.888886427 ; 452154.592091888 ; 2014 ; 923053 ; 1446.8235294117646 ; Grasland, blijvend ; ['Hn21'] ; [0.54522, 0.5181, 0.546569, 0.588419, 0.623419]
200 ; 221430.888886427 ; 452154.592091888 ; 2013 ; 1605661 ; 1446.8235294117646 ; Grasland, blijvend ; ['Hn21'] ; [0.445581, 0.5469, 0.426206, 0.422125, 0.432519]
200 ; 221430.888886427 ; 452154.592091888 ; 2012 ; 2993464 ; 1446.8235294117646 ; Grasland, blijvend ; ['Hn21'] ; []
200 ; 221430.888886427 ; 452154.592091888 ; 2015 ; 3579807 ; 1446.8235294117646 ; Grasland, blijvend ; ['Hn21'] ; [0.506481, 0.522856, 0.469456, 0.515144, 0.633563]
200 ; 221430.888886427 ; 452154.592091888 ; 2017 ; 4596951 ; 1446.8235294117646 ; Grasland, blijvend ; ['Hn21'] ; [0.549, 0.654, 0.462, 0.405, 0.555]
201 ; 220751.820375517 ; 451676.998079961 ; 2016 ; 689920 ; NA ; Bos, blijvend, m

209 ; 220507.07422912 ; 451818.503170547 ; 2013 ; 2304066 ; 1372.2857142857142 ; Mais, snij- ; ['Hn21'] ; [0.219933, 0.2789, 0.271167, 0.282967, 0.273267]
209 ; 220507.07422912 ; 451818.503170547 ; 2012 ; 3059294 ; 1369.1666666666667 ; Mais, snij- ; ['Hn21'] ; []
209 ; 220507.07422912 ; 451818.503170547 ; 2015 ; 3891023 ; 1372.2857142857142 ; Grasland, tijdelijk ; ['Hn21'] ; [0.465233, 0.504367, 0.413733, 0.555033, 0.474333]
209 ; 220507.07422912 ; 451818.503170547 ; 2017 ; 4277532 ; 1372.2857142857142 ; Grasland, tijdelijk ; ['Hn21'] ; [0.417, 0.498, 0.645, 0.561, 0.543]
210 ; 221176.577487563 ; 450678.689148285 ; 2016 ; 645970 ; 1520.1290322580646 ; Grasland, tijdelijk ; ['Hn21'] ; [0.48779, 0.580137, 0.596226, 0.566232, 0.521716]
210 ; 221176.577487563 ; 450678.689148285 ; 2014 ; 1533009 ; 1518.7 ; Mais, snij- ; ['Hn21'] ; [0.355426, 0.302889, 0.319605, 0.297116, 0.319811]
210 ; 221176.577487563 ; 450678.689148285 ; 2013 ; 1689162 ; 1518.7 ; Grasland, tijdelijk ; ['Hn21'] ; [0.47609

220 ; 220256.737729757 ; 451748.734443542 ; 2017 ; 4706334 ; 1389.764705882353 ; Grasland, blijvend ; ['Hn21'] ; [0.513, 0.645, 0.663, 0.669, 0.657]
221 ; 221158.240955551 ; 452018.943298215 ; 2016 ; 18246 ; 1464.7 ; Grasland, blijvend ; ['cHn21', 'Hn21', 'pZg23'] ; [0.513622, 0.596726, 0.611056, 0.608867, 0.538744]
221 ; 221158.240955551 ; 452018.943298215 ; 2014 ; 1297231 ; 1464.7 ; Grasland, blijvend ; ['cHn21', 'Hn21', 'pZg23'] ; [0.565818, 0.57507, 0.606237, 0.616874, 0.646126]
221 ; 221158.240955551 ; 452018.943298215 ; 2013 ; 1865327 ; 1464.7 ; Grasland, blijvend ; ['cHn21', 'Hn21', 'pZg23'] ; [0.438, 0.577359, 0.543448, 0.539744, 0.5629]
221 ; 221158.240955551 ; 452018.943298215 ; 2012 ; 2845986 ; 1464.7 ; Grasland, blijvend ; ['cHn21', 'Hn21', 'pZg23'] ; []
221 ; 221158.240955551 ; 452018.943298215 ; 2015 ; 3872186 ; 1464.878048780488 ; Grasland, blijvend ; ['cHn21', 'Hn21', 'pZg23'] ; [0.478232, 0.521979, 0.449457, 0.479539, 0.584339]
221 ; 221158.240955551 ; 452018.943298215

230 ; 222104.137214342 ; 450857.54491257 ; 2012 ; 3197084 ; 1576.4285714285713 ; Grasland, blijvend ; ['Hn21'] ; []
230 ; 222104.137214342 ; 450857.54491257 ; 2015 ; 3578155 ; 1576.4285714285713 ; Grasland, blijvend ; ['Hn21'] ; [0.275567, 0.329733, 0.429267, 0.368333, 0.393733]
230 ; 222104.137214342 ; 450857.54491257 ; 2017 ; 4380603 ; 1576.4285714285713 ; Grasland, blijvend ; ['Hn21'] ; [0.447, 0.534, 0.606, 0.612, 0.597]
231 ; 223472.760000377 ; 451358.54821056 ; 2016 ; 225560 ; 1669.1538461538462 ; Grasland, tijdelijk ; ['Hn21'] ; [0.492257, 0.545741, 0.580494, 0.639216, 0.56861]
231 ; 223472.760000377 ; 451358.54821056 ; 2014 ; 1367895 ; 1669.1538461538462 ; Grasland, blijvend ; ['Hn21'] ; [0.532929, 0.510127, 0.548381, 0.608112, 0.644496]
231 ; 223472.760000377 ; 451358.54821056 ; 2013 ; 2375834 ; 1668.867469879518 ; Mais, snij- ; ['Hn21'] ; [0.280734, 0.373941, 0.3783, 0.324995, 0.329497]
231 ; 223472.760000377 ; 451358.54821056 ; 2012 ; 3096564 ; 1668.867469879518 ; Mais, snij

240 ; 223085.036415601 ; 451828.920422334 ; 2016 ; 171884 ; 1669.0 ; Bos, blijvend, met herplantplicht ; ['zEZ21'] ; []
240 ; 223085.036415601 ; 451828.920422334 ; 2014 ; 1001516 ; 1669.0 ; Bos, blijvend, met herplantplicht ; ['zEZ21'] ; []
240 ; 223085.036415601 ; 451828.920422334 ; 2013 ; 1708537 ; 1669.0 ; Bos, blijvend, met herplantplicht ; ['zEZ21'] ; []
240 ; 223085.036415601 ; 451828.920422334 ; 2012 ; 2928883 ; 1669.0 ; Bos, blijvend, met herplantplicht ; ['zEZ21'] ; []
240 ; 223085.036415601 ; 451828.920422334 ; 2015 ; 3337374 ; 1669.0 ; Bos, blijvend, met herplantplicht ; ['zEZ21'] ; []
240 ; 223085.036415601 ; 451828.920422334 ; 2017 ; 4243677 ; 1669.0 ; Bos, blijvend, met herplantplicht ; ['zEZ21'] ; [0.315, 0.555, 0.666, 0.675, 0.633]
